In [1]:
# PyTorch stuff
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

# Misc. Imports
from tqdm import tqdm
import pandas as pd
import numpy as np

# STEAD
from stead import *

In [2]:
# Locations of STEAD Dataset
npy_file = '/scratch/cs101/STEAD/stead_full.npy'
csv_file = '/scratch/cs101/STEAD/stead_metadata_new.csv'

In [3]:
# Device settings for PyTorch
# If using 'cpu', set 'device_idx' to 0
# If using 'cuda', set 'device_idx' to an unused device (check nvidia-smi)
device_type = 'cuda'
device_idx  = 0
device = torch.device(device_type, device_idx)

In [4]:
# Set up the impulse signal
impulse = SquareImpulse(10, 1)

In [5]:
# Load the full dataset
full_dataset = STEADDataset(csv_file, npy_file, impulse)
# Filter out non-earthquake signals
full_dataset.filter(lambda df: df['trace_category'] == 'earthquake_local')
# Create train/test split
train, test = full_dataset.split(0.8)

Loading metadata...
Reading trace files...
Loading data into memory...
Done.


In [6]:
# Just a mock neural net
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        
        self.conv_ds1 = nn.Conv1d(in_channels=3, out_channels=8, kernel_size=11, padding=5)
        self.conv_ds2 = nn.Conv1d(in_channels=8, out_channels=16, kernel_size=7, padding=3)
        self.conv_ds3 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5, padding=2)
        self.conv_ds4 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        
        self.conv_us_p1 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, padding=1)
        self.conv_us_p2 = nn.Conv1d(in_channels=32, out_channels=16, kernel_size=5, padding=2)
        self.conv_us_p3 = nn.Conv1d(in_channels=16, out_channels=8, kernel_size=7, padding=3)
        self.conv_us_p4 = nn.Conv1d(in_channels=8, out_channels=1, kernel_size=11, padding=5)
        
        self.conv_us_s1 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, padding=1)
        self.conv_us_s2 = nn.Conv1d(in_channels=32, out_channels=16, kernel_size=5, padding=2)
        self.conv_us_s3 = nn.Conv1d(in_channels=16, out_channels=8, kernel_size=7, padding=3)
        self.conv_us_s4 = nn.Conv1d(in_channels=8, out_channels=1, kernel_size=11, padding=5)
        
        self.conv_us_c1 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, padding=1)
        self.conv_us_c2 = nn.Conv1d(in_channels=32, out_channels=16, kernel_size=5, padding=2)
        self.conv_us_c3 = nn.Conv1d(in_channels=16, out_channels=8, kernel_size=7, padding=3)
        self.conv_us_c4 = nn.Conv1d(in_channels=8, out_channels=1, kernel_size=11, padding=5)
                
        self.mp1 = nn.MaxPool1d(2)
        
        self.us1 = nn.Upsample(scale_factor=2)
            
    def forward(self, x):
        # Combined encoder
        x = self.conv_ds1(x)
        x = self.mp1(x)
        x = self.conv_ds2(x)
        x = self.mp1(x)
        x = self.conv_ds3(x)
        x = self.mp1(x)
        x = self.conv_ds4(x)
        x = self.mp1(x)
        
        # P-arrival Decoder
        p = self.us1(x)
        p = self.conv_us_p1(p)
        p = self.us1(p)
        p = self.conv_us_p2(p)
        p = self.us1(p)
        p = self.conv_us_p3(p)
        p = self.us1(p)
        p = self.conv_us_p4(p)
        
        # S-arrival Decoder
        s = self.us1(x)
        s = self.conv_us_s1(s)
        s = self.us1(s)
        s = self.conv_us_s2(s)
        s = self.us1(s)
        s = self.conv_us_s3(s)
        s = self.us1(s)
        s = self.conv_us_s4(s)
        
        # Coda Decoder
        c = self.us1(x)
        c = self.conv_us_c1(c)
        c = self.us1(c)
        c = self.conv_us_c2(c)
        c = self.us1(c)
        c = self.conv_us_c3(c)
        c = self.us1(c)
        c = self.conv_us_c4(c)
        
        return p, s, c

In [8]:
# Set up the PyTorch DataLoaders
train_data = data.DataLoader(train, batch_size=1000, shuffle=True, num_workers=4)
test_data = data.DataLoader(test, batch_size=1000, shuffle=True, num_workers=4)

In [9]:
# TensorBoard writer
writer = SummaryWriter()

In [10]:
# Initialize the network, optimization criterion, and optimizer
net = Network().to(device)
criterion_p = nn.BCEWithLogitsLoss().to(device)
criterion_s = nn.BCEWithLogitsLoss().to(device)
criterion_c = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

In [ ]:
num_epochs = 20

# Each iteration is one full pass through the data
for e in range(num_epochs):
    print(f'Epoch {e}:', end='')
    
    loss_total = 0
    
    # Run through the training set
    for idx, batch in enumerate(tqdm(train_data)):
        # Get inputs and labels, and move them to device
        inputs, labels = batch
        inputs = inputs.to(device)
        labels = [label.to(device) for label in labels]
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Make predictions
        (p_pred, s_pred, c_pred) = net(inputs)
        
        # Compute loss
        p_loss = criterion_p(p_pred, labels[0])
        s_loss = criterion_s(s_pred, labels[0])
        c_loss = criterion_c(c_pred, labels[0])
        loss = p_loss + s_loss + c_loss
        
        # Make optimization step
        loss.backward()
        optimizer.step()
        
        # Log loss to TensorBoard
        writer.add_scalar('Loss/Batch/Train', loss.item(), e * len(train_data) + idx)
        loss_total += loss.item()
    
    # Log whole-epoch loss average
    writer.add_scalar('Loss/Train', loss_total / len(train_data), e)
    
    loss_total = 0
    
    # Run through the test set
    with torch.no_grad():
        for batch in tqdm(test_data):
            # Get inputs and labels, and move them to device
            inputs, labels = batch
            inputs = inputs.to(device)
            labels = [label.to(device) for label in labels]
            
            # Make predictions
            (p_pred, s_pred, c_pred) = net(inputs)

            # Compute loss
            p_loss = criterion_p(p_pred, labels[0])
            s_loss = criterion_s(s_pred, labels[0])
            c_loss = criterion_c(c_pred, labels[0])
            loss = p_loss + s_loss + c_loss
            loss_total += loss.item()

    # Log loss to TensorBoard
    writer.add_scalar('Loss/Test', loss_total / len(test_data), e)

  0%|          | 0/825 [00:00<?, ?it/s]

Epoch 0:

 66%|██████▌   | 136/207 [00:17<00:08,  8.52it/s]